In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt

np.random.seed(2)
tf.random.set_seed(2)

# creating classifier

In [ ]:
autoencoder_model = tf.keras.models.load_model(
    r"./saved_models/encoder_v3",
    custom_objects={"custom_loss": None},
)

autoencoder_model.trainable = False
model_input = autoencoder_model.layers[0].input
encoder_output = autoencoder_model.get_layer("encoder_output").output
encoder = keras.Model(inputs=[model_input], outputs=[encoder_output])

x = x_in = keras.layers.Input((256, 256, 3))

encoder_output = encoder(x)

"""
x = keras.layers.Conv2D(
    8,
    kernel_size=3,
    strides=(2, 2),
    padding="same",
)(encoder_output)
"""

x = keras.layers.Flatten()(encoder_output)
x = keras.layers.Dropout(0.4)(x)
x = keras.layers.Dense(5, "softmax")(x)

# create model
classifier = keras.Model(inputs=[x_in], outputs=[x])

classifier.summary()

# load classifier

In [ ]:
classifier = tf.keras.models.load_model(r"./saved_models/classifier_v2")
classifier.summary(expand_nested=True)

In [ ]:
for l in classifier.layers:
    if isinstance(l, tf.keras.Model):
        for l2 in l.layers:
            print(l2)

    print(l.name)

# Save model weights as numpy file

the name of the file indicates the 'type' of the layer, the layer 'name', and 'activation', separeted by '.'.\
eg: Conv2D.encoder_output.Linear

In [ ]:
def save_layer_file(id, layer, path):
    layer_name = layer.name
    layer_type = None
    activation_type = "Linear"
    weights, bias = [[], []]
    
    layer_config = layer.get_config()

    if isinstance(layer, tf.keras.layers.Conv2D):
        layer_type = "Conv2D"
        weights, bias = layer.get_weights()
        activation_type = layer_config["activation"]

    elif isinstance(layer, tf.keras.layers.DepthwiseConv2D):
        layer_type = "DepthwiseConv2D"
        weights, bias = layer.get_weights()
        activation_type = layer.get_config()["activation"]

    elif isinstance(layer, tf.keras.layers.Dense):
        layer_type = "Dense"
        weights, bias = layer.get_weights()
        activation_type = layer.get_config()["activation"]

    elif isinstance(layer, tf.keras.layers.Flatten):
        layer_type = "Flatten"

    elif isinstance(layer, tf.keras.layers.InputLayer):
        layer_type = "InputLayer"

    elif isinstance(layer, tf.keras.layers.Dropout):
        layer_type = "Dropout"

    else:
        print(f"layer type {type(layer)} not expected..")

    weight_path = f"{id}.{layer_type}.{layer_name}.{activation_type}.weights.npy"
    bias_path = f"{id}.{layer_type}.{layer_name}.{activation_type}.bias.npy"

    np.save(
        f"{path}/" + weight_path,
        weights,
    )
    np.save(
        f"{path}/" + bias_path,
        bias,
    )

    layerJson = {
        "layerName": layer_name,
        "layerType": layer_type,
        "activation": activation_type,
        "weightPath": weight_path,
        "biasPath": bias_path,
        "layer_config" : layer_config
    }

    return layerJson



save_path = f"./saved_models/classifier_v2"
modelJson = {
    "modelName": "classifier_v2",
    "layers": [],
}


index = 0
for l in classifier.layers:
    if isinstance(l, tf.keras.Model):
        for l2 in l.layers[1:]:
            layerJson = save_layer_file(index, l2, save_path)
            modelJson["layers"].append(layerJson)
            index += 1
    else:
        print(l.name)
        layerJson = save_layer_file(index, l, save_path)
        modelJson["layers"].append(layerJson)
        index += 1

import json

with open(f"{save_path}/model.json", "w") as file:
    file.write(json.dumps(modelJson))
    print("model exported ...")

# create the numpy dense/softmax layer

In [ ]:
from Layers import Dense
from Activations import Softmax

softmax = Softmax()
fc = Dense("dense", activation=softmax)
fc.load_weights(
    "/home/george/Desktop/dnn_processor/reference_algorithm/neural_network/saved_models/classifier_v2/6.Dense.dense.softmax.weights.npy",
    "/home/george/Desktop/dnn_processor/reference_algorithm/neural_network/saved_models/classifier_v2/6.Dense.dense.softmax.bias.npy",
)

# test dense/softmax layer

In [ ]:
# the tensorflow layer and the numpy layer on the same input

denseLayer = classifier.get_layer("dense")

input_test = np.ones((32, denseLayer.get_weights()[0].shape[0]))

o_dut = fc(input_test, False).astype(np.float16).ravel()
o_dur = denseLayer(input_test).numpy().astype(np.float16).ravel()

np.savetxt("dut_fc.txt", o_dut)
np.savetxt("dur_fc.txt", o_dur)

assert o_dut.shape == o_dur.shape, f"shape test failed. { o_dut.shape} != {o_dur.shape}"
assert np.all(o_dut == o_dur), f"val test failed. Check dut_fc and dur_fc .txt files for more details."


print("test_passed!")

# create conv2D numpy layer

In [ ]:
from Layers import Conv2D
from Activations.ActivationFactory import activation_from_name

conv2D_ref = classifier.layers[1].get_layer("conv2d")
input_test = np.ones((1, 13, 13, conv2D_ref.weights[0].shape[-2]))


npConv2D = Conv2D(
    strides=conv2D_ref.get_config()["strides"],
    padding=conv2D_ref.get_config()["padding"],
    activation=activation_from_name(conv2D_ref.get_config()["activation"]),
)
npConv2D.load_weights(
    weight_path="./saved_models/classifier_v2/2.Conv2D.conv2d.tanh.weights.npy",
    bias_path="./saved_models/classifier_v2/2.Conv2D.conv2d.tanh.bias.npy",
)


assert np.all(conv2D_ref.weights[0] == npConv2D.weights), "weights are different"
assert np.all(conv2D_ref.weights[1] == npConv2D.bias), "bias are different"
print("weights loaded correctly...")

# test conv2D layer

In [ ]:
o_dut = npConv2D(input_test, training=False).astype(np.float16)
o_dur = conv2D_ref(input_test).numpy().astype(np.float16)

np.savetxt("dut_conv.txt", o_dut.ravel())
np.savetxt("dur_conv.txt", o_dur.ravel())

assert o_dut.shape == o_dur.shape, f"shape test failed. { o_dut.shape} != {o_dur.shape}"
assert np.all(
    o_dut == o_dur
), f"val test failed. Check dut_conv.txt and dur_conv.txt files. "

print("test passed!")

# create depthwise convolution

In [ ]:
from Layers import DephtwiseConv2D

depthwise_conv2d_ref = classifier.layers[1].get_layer("depthwise_conv2d")

npDephtwiseConv2D = DephtwiseConv2D(
    strides=depthwise_conv2d_ref.get_config()["strides"],
    padding=depthwise_conv2d_ref.get_config()["padding"],
    activation=activation_from_name(depthwise_conv2d_ref.get_config()["activation"]),
)
npDephtwiseConv2D.load_weights(
    weight_path="./saved_models/classifier_v2/1.DepthwiseConv2D.depthwise_conv2d.linear.weights.npy",
    bias_path="./saved_models/classifier_v2/1.DepthwiseConv2D.depthwise_conv2d.linear.bias.npy",
)

assert np.all(
    depthwise_conv2d_ref.weights[0] == npDephtwiseConv2D.weights
), "weights are different"
assert np.all(
    depthwise_conv2d_ref.weights[1] == npDephtwiseConv2D.bias
), "bias are different"
print("weights loaded correctly...")

# test depthwise convolution

In [ ]:
input_test = np.ones((1, 32, 32, depthwise_conv2d_ref.weights[0].shape[-2]))

o_dut = npDephtwiseConv2D(input_test, training=False).astype(np.float16)
o_dur = depthwise_conv2d_ref(input_test).numpy().astype(np.float16)

print(o_dut)
print(o_dur)

np.savetxt("dut_dconv.txt", (o_dut.ravel()*100).astype(np.int32), fmt="%d")
np.savetxt("dur_dconv.txt", (o_dur.ravel()*100).astype(np.int32), fmt="%d")

assert o_dut.shape == o_dur.shape, f"shape test failed. { o_dut.shape} != {o_dur.shape}"
assert np.all(
    o_dut == o_dur
), f"val test failed. Check dut_dconv.txt and dur_dconv.txt files "

print("test passed!")

# create numpy model

In [ ]:
import json
from NumpyModel import Model
from Layers import InputLayer, Conv2D, Dense, DephtwiseConv2D, Flatten
from Activations.ActivationFactory import activation_from_name
import numpy as np


classifier_model_path = "/home/george/Desktop/dnn_processor/reference_algorithm/neural_network/saved_models/classifier_v2/"

modelJson = json.load(open(classifier_model_path + "model.json"))

model = Model()


for layerConfig in modelJson["layers"]:
    if layerConfig["layerType"] == "InputLayer":
        layer = InputLayer(layerConfig["layerName"])

    elif layerConfig["layerType"] == "Conv2D":
        layer = Conv2D(
            padding=layerConfig["layer_config"]["padding"],
            strides=layerConfig["layer_config"]["strides"],
            name=layerConfig["layerName"],
        )
        layer.load_weights(
            classifier_model_path + layerConfig["weightPath"],
            classifier_model_path + layerConfig["biasPath"],
        )

        layer.activation = activation_from_name(layerConfig["activation"])

    elif layerConfig["layerType"] == "DepthwiseConv2D":
        layer = DephtwiseConv2D(
            padding=layerConfig["layer_config"]["padding"],
            strides=layerConfig["layer_config"]["strides"],
            name=layerConfig["layerName"],
        )
        layer.load_weights(
            classifier_model_path + layerConfig["weightPath"],
            classifier_model_path + layerConfig["biasPath"],
        )
        layer.activation = activation_from_name(layerConfig["activation"])

    elif layerConfig["layerType"] == "Dense":
        layer = Dense(layerConfig["layerName"])
        layer.load_weights(
            classifier_model_path + layerConfig["weightPath"],
            classifier_model_path + layerConfig["biasPath"],
        )
        layer.activation = activation_from_name(layerConfig["activation"])

    elif layerConfig["layerType"] == "Flatten":
        layer = Flatten(layerConfig["layerName"])

    elif layerConfig["layerType"] == "Dropout":
        print("ignoring not implemented dropout")
        continue

    else:
        print(layerConfig)
        raise NotImplementedError(layerConfig["layerName"])

    model.layers.append(layer)

print(model.layers)

In [ ]:
import tensorflow as tf
from tensorflow import keras

np.random.seed(2)
tf.random.set_seed(2)

classifier = tf.keras.models.load_model(r"./saved_models/classifier_v2")
classifier.summary(expand_nested=True)

# test numpy model

In [ ]:
print(classifier.layers[0].get_config()["batch_input_shape"])
input_test = np.zeros(
    shape=[
        1,
    ]
    + list(classifier.layers[0].get_config()["batch_input_shape"][1:])
)


print(classifier(input_test).numpy())
print(model(input_test, training=False).astype(np.float16))


o_dut_model = classifier(input_test).numpy()
o_ref_model = model(input_test, training=False).astype(np.float16)

assert np.all(o_dut_model == o_ref_model), "val test failed. Check values"